In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from keras import Sequential
from sklearn.metrics import f1_score, accuracy_score
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

In [4]:
train_data = keras.utils.image_dataset_from_directory(
    directory = '/content/drive/MyDrive/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

test_data = keras.utils.image_dataset_from_directory(
    directory = '/content/drive/MyDrive/test',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

Found 557 files belonging to 2 classes.
Found 140 files belonging to 2 classes.


In [5]:
def imageprocessor(image,label):
  image=tf.cast(image/255.,tf.float32)
  return image,label

train_data=train_data.map(imageprocessor)
test_data=test_data.map(imageprocessor)



In [6]:
model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 254, 254, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 batch_normalization_1 (Bat  (None, 125, 125, 64)      256       
 chNormalization)                                                
                                                        

In [7]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [8]:
model.fit(train_data,epochs=20,validation_data=test_data)

Epoch 1/20
18/18 [==============================] - 54s 1s/step - loss: 6.2809 - accuracy: 0.5476 - val_loss: 3.1194 - val_accuracy: 0.4786
Epoch 2/20
18/18 [==============================] - 6s 262ms/step - loss: 4.2842 - accuracy: 0.5961 - val_loss: 3.7373 - val_accuracy: 0.5286
Epoch 3/20
18/18 [==============================] - 4s 166ms/step - loss: 3.2858 - accuracy: 0.6589 - val_loss: 6.1108 - val_accuracy: 0.4786
Epoch 4/20
18/18 [==============================] - 6s 216ms/step - loss: 3.0770 - accuracy: 0.6697 - val_loss: 3.2995 - val_accuracy: 0.5286
Epoch 5/20
18/18 [==============================] - 4s 168ms/step - loss: 2.3529 - accuracy: 0.6732 - val_loss: 4.7334 - val_accuracy: 0.5000
Epoch 6/20
18/18 [==============================] - 5s 197ms/step - loss: 1.3349 - accuracy: 0.7738 - val_loss: 5.3616 - val_accuracy: 0.5071
Epoch 7/20
18/18 [==============================] - 5s 204ms/step - loss: 1.2063 - accuracy: 0.8007 - val_loss: 8.7560 - val_accuracy: 0.5214
Epoch 8/

In [9]:
y_true = []
y_prob = []

for images, labels in test_data:
    y_true.extend(labels.numpy())
    y_prob.extend(model.predict(images))

y_true = np.array(y_true)
y_prob = np.array(y_prob)
y_pred = (y_prob > 0.5).astype(int)

1/1 [==============================] - 0s 183ms/step


In [11]:
y_pred = (y_prob > 0.5).astype(int)
f1 = f1_score(y_true, y_pred)
print(f1)


0.4814814814814815
